In [1]:
from google.colab import drive
drive.mount('/content/drive/')

!git clone https://github.com/realcrane/iPLAN-Interactive-and-Procedural-Layout-Planning

import os
print("Directory di lavoro corrente:", os.getcwd())
roomType_directory = '/content/iPLAN-Interactive-and-Procedural-Layout-Planning/'
os.chdir(roomType_directory)
print("Nuova directory di lavoro:", os.getcwd())

Mounted at /content/drive/
Cloning into 'iPLAN-Interactive-and-Procedural-Layout-Planning'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 301 (delta 96), reused 106 (delta 46), pack-reused 91
Receiving objects: 100% (301/301), 12.98 MiB | 27.92 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Directory di lavoro corrente: /content
Nuova directory di lavoro: /content/iPLAN-Interactive-and-Procedural-Layout-Planning


In [2]:
import numpy as np
import torch
import scipy.io as sio

In [26]:
def getList(path):
  mat_list = os.listdir(path)
  mat_list.sort()
  temp_list = []
  for name in mat_list[:]:
      mat_path = os.path.join(path, name)
      data = sio.loadmat(mat_path, squeeze_me=True, struct_as_record=False)['data']
      if len(data.rCenters) == 0:
          temp_list.append(name)
  return temp_list

In [ ]:
def check_rcenters(center, rCenters):
    flag = True
    center = np.array(center)
    for i in range(len(rCenters)):
        temp = np.array(rCenters[i])
        dist = np.linalg.norm(center - temp)
        if dist < 9:
            flag = False
            return flag
    return flag

In [ ]:
def find_center(predict, r_t, mask_size=4):
    index_point = []
    index = np.where(predict == r_t.cpu().numpy())
    for ind in range(index[0].shape[0]):
        index_point.append((index[0][ind], index[1][ind]))

    num_point = 0
    if len(index_point) > 0:
        min_h = np.min(index[0])
        min_w = np.min(index[1])
        max_h = np.max(index[0])
        max_w = np.max(index[1])

        if max_h - min_h + 1 <= 2 * mask_size and max_w - min_w + 1 <= 2 * mask_size:
            predict_h = (min_h + max_h) // 2
            predict_w = (min_w + max_w) // 2
            num_point = len(index_point)
        else:
            predict_h, predict_w = index_point[0]
            for point in index_point:
                new_num_point = 0
                for other_point in index_point:
                    if abs(other_point[0] - point[0]) <= mask_size and abs(
                            other_point[1] - point[1]) <= mask_size:
                        new_num_point += 1
                if new_num_point > num_point:
                    predict_h, predict_w = point
                    num_point = new_num_point
    if num_point > 0:
        return np.array([predict_h, predict_w])
    else:
        return np.array([0, 0])

In [5]:
def get_rcenters(fp, location_model, location_connect, location_embedding, mask_size=4):
    pred_rCenters = []
    pred_rTypes = []

    living_node = fp.living_node
    h, w = living_node['centroid']
    pred_rCenters.append(np.array([int(h), int(w)]))
    pred_rTypes.append(0)

    input_location = fp.get_composite_location(num_extra_channels=0)
    continue_rTypes = fp.continue_rTypes.squeeze()
    input_location = input_location.unsqueeze(0).cuda()

    room_num = continue_rTypes.shape[0]

    iter = 0
    flag = True
    while room_num > 0:
        update_id = np.zeros(room_num, dtype=bool)
        iter = iter + 1

        np.random.shuffle(continue_rTypes)
        with torch.no_grad():
            for n in range(room_num):
                score_model = location_model(input_location)
                r_t = torch.LongTensor(continue_rTypes[n:n + 1]).cuda()
                one_hot_label = torch.nn.functional.one_hot(r_t, num_classes=13)
                score_embedding = location_embedding(one_hot_label.float())
                score_temp = torch.cat([score_model, score_embedding], 1)
                score_connect = location_connect(score_temp)

                score_softmax = torch.softmax(score_connect, dim=1)
                output = score_softmax.cpu().numpy()
                predict = np.argmax(output, axis=1)[0]

                center = find_center(predict, r_t)
                h, w = center

                if h == 0 or w == 0 or input_location[0, 0, h, w] == 0:
                    continue

                flag = check_rcenters(center, pred_rCenters)

                if flag:
                    pred_rCenters.append(np.array([h, w]))
                    pred_rTypes.append(r_t.cpu().numpy().reshape(1)[0])
                    update_id[n] = True
                    min_h = max(h - mask_size, 0)
                    max_h = min(h + mask_size, 128 - 1)
                    min_w = max(w - mask_size, 0)
                    max_w = min(w + mask_size, 128 - 1)
                    input_location[0, r_t + 4, min_h:max_h + 1, min_w:max_w + 1] = 1.0
                    input_location[0, 3, :, :] = input_location[0, 4:, :, :].sum(0)

                    iter = 0

        continue_rTypes = continue_rTypes[~update_id]
        room_num = continue_rTypes.shape[0]

        if iter > 5:
            if room_num == 0:  # 0: all room centers are located, 1: one room missing, 2: two room missing
                flag = True
            else:
                flag = False
            break

    return pred_rCenters, pred_rTypes, flag

In [6]:
from synth.floorplan.roomlocation_fp import FloorPlan
from room_location.Living import models as living
from room_location.Location import models as location

In [7]:
path_model_roomLocation_resnet = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/resnet18-5c106cde.pth"

path_model_roomLocation_location_resnet = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_resnet18_100.pth"
path_model_roomLocation_location_up = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_up1_100.pth"
path_model_roomLocation_location_embed = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_embed_100.pth"

path_model_roomLocation_living_fci = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/living_fc1_300.pth"
path_model_roomLocation_living_resnet = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/living_resnet18_300.pth"

In [8]:
# step1: configure model
print('Building model...')
living_model = living.model(
    module_name="living",
    model_name="resnet18_fc1",
    input_channel=3,
    output_channel=2,
    pretrained=False,
)
living_connect = living.connect(
    module_name="living",
    model_name="resnet18_fc1",
    input_channel=512,
    output_channel=2,
    reshape=True
)

load_living_model_path = path_model_roomLocation_living_resnet
load_living_connect_path = path_model_roomLocation_living_fci
living_model.load_model(load_living_model_path)
living_connect.load_model(load_living_connect_path)
living_model.cuda()
living_connect.cuda()

location_model = location.model(
    module_name='location',
    model_name='resnet18_up1',
    input_channel=13 + 4,
    output_channel=13 + 3,
    pretrained=True,
    pretrained_path= path_model_roomLocation_resnet
)

location_connect = location.connect(
    module_name='location',
    model_name='resnet18_up1',
    input_channel=512,
    output_channel=13 + 3,
    reshape=False
)
location_embedding = location.embedding(
    module_name='location',
    model_name='resnet18_up1',
    input_channel=13,
    output_channel=256,
    reshape=False
)
epoch = 100
load_location_model_path = path_model_roomLocation_location_resnet
load_location_connect_path = path_model_roomLocation_location_up
load_location_embedding_path = path_model_roomLocation_location_embed

location_model.load_model(load_location_model_path)
location_connect.load_model(load_location_connect_path)
location_embedding.load_model(load_location_embedding_path)
location_model.cuda()
location_connect.cuda()
location_embedding.cuda()

living_model.eval()
living_connect.eval()
location_model.eval()
location_connect.eval()
location_embedding.eval()

# step2: data
print('Building dataset...')
data_root = "/content/iPLAN-Interactive-and-Procedural-Layout-Planning/data/test"
floorplans = getList(data_root)
print('The length of testing data is {}'.format(len(floorplans)))

Building model...
load the pretrained model...
Building dataset...
The length of testing data is 2


In [9]:
# step3: testing
print('Starting to Testing...')
total = 0
Data = []
for i in range(len(floorplans)):
  fp_name = floorplans[i]
  print(fp_name)
  fp_path = os.path.join(data_root, fp_name)
  print(fp_path)
  fp = FloorPlan(fp_path)

  input_living = fp.get_composite_living()
  input_living = input_living.unsqueeze(0).cuda()

  with torch.no_grad():
    score_model = living_model(input_living)
    score_connect_living = living_connect(score_model)
    living_center = torch.round(score_connect_living)
    living_center = living_center.cpu().squeeze().numpy()

  node = {}
  node['category'] = int(0)
  x = living_center[0]
  y = living_center[1]
  node['centroid'] = (int(x), int(y))
  fp.add_room(node)
  fp.living_node = node

  iteration = 0
  # the maximum sampling for room location
  while iteration < 50:
    iteration = iteration + 1
    pred_rCenters, pred_rTypes, flag = get_rcenters(fp, location_model, location_connect, location_embedding)
    if flag:
      fp.update_rCenters(pred_rCenters, pred_rTypes)
      data = fp.to_dict()
      Data.append(data)
      sio.savemat(fp_path, {'data': data})
      total = total + 1
      print('{}/{}, has processd {} '.format(i + 1, total, data['name']))
      break

print(f'{total} layouts')



Starting to Testing...
0.mat
/content/iPLAN-Interactive-and-Procedural-Layout-Planning/data/test/0.mat
1/1, has processd 0 
1.mat
/content/iPLAN-Interactive-and-Procedural-Layout-Planning/data/test/1.mat
2/2, has processd 1 
2 layouts


In [10]:
Data

[{'name': '0',
  'gt_rTypes': array([0, 1, 2, 3, 7, 9]),
  'gt_rBoxes': array([[ 14,  55, 101,  92],
         [ 80,  36, 110,  62],
         [ 26,  57,  41,  72],
         [ 65,  36,  78,  53],
         [ 43,  36,  63,  62],
         [103,  64, 114,  92]]),
  'Boundary': array([[ 19,  82,   1,   1],
         [ 27,  82,   1,   1],
         [ 43,  82,   0,   0],
         [ 43,  92,   1,   0],
         [114,  92,   2,   0],
         [114,  64,   3,   0],
         [110,  64,   2,   0],
         [110,  36,   3,   0],
         [ 43,  36,   0,   0],
         [ 43,  57,   3,   0],
         [ 26,  57,   0,   0],
         [ 26,  74,   3,   0],
         [ 14,  74,   0,   0],
         [ 14,  82,   1,   0]]),
  'rTypes': array([0]),
  'rBoxes': array([], dtype=int64),
  'rCenters': array([[58, 71]])},
 {'name': '1',
  'gt_rTypes': array([0, 1, 2, 3, 5, 7, 9]),
  'gt_rBoxes': array([[ 28,  22,  65, 106],
         [ 67,  72,  97, 106],
         [ 25,  22,  42,  36],
         [ 28,  38,  36,  50],
   

In [25]:
for key, value in data.items():
  print(key, value)

name 1
gt_rTypes [0 1 2 3 5 7 9]
gt_rBoxes [[ 28  22  65 106]
 [ 67  72  97 106]
 [ 25  22  42  36]
 [ 28  38  36  50]
 [ 67  52  97  70]
 [ 67  22  97  50]
 [ 99  72 103 106]]
Boundary [[ 59 101   0   1]
 [ 59 106   1   0]
 [103 106   2   0]
 [103  72   3   0]
 [ 97  72   2   0]
 [ 97  22   3   0]
 [ 25  22   0   0]
 [ 25  36   1   0]
 [ 28  36   0   0]
 [ 28 100   1   0]
 [ 59 100   0   0]]
rTypes [0]
rBoxes []
rCenters [[64 69]]
